In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/'
!ls

/content/gdrive/.shortcut-targets-by-id/1-WHOINXFZ4JSjL7g5g092rZ8rCZfdBPw/odir-ivashnyov/ODIR
CrossValidationScore.py		  Predictions.py
cv_split.pickle			  predict.py
data				  __pycache__
EDA.ipynb			  README.md
initial_test_b3_fold3_clahe	  split_folds.ipynb
logs				  submit
logs_final			  submit_final
MNIST				  submit_initial_test_b3.csv
model_concate_10_no_epoch.txt	  submit_oanh_initial_test_b3.csv
model_concate_15epoch_weight.txt  submit_vessel_initial_test_b3.csv
model_concate_19epoch.txt	  train_1cycle.py
model_concate_20epoch.txt	  train_all_folds_args.sh
model_concate_5_no_epoch.txt	  train_all_folds.sh
model_concate_6epoch.txt	  train.py
model_concate_epoch_20.txt	  utils.py
odir_submit.py			  Visualization.ipynb


In [3]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

     |████████████████████████████████| 358kB 6.9MB/s 
     |████████████████████████████████| 2.1MB 23.4MB/s 
     |████████████████████████████████| 317kB 36.3MB/s 
     |████████████████████████████████| 163kB 38.9MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


     |████████████████████████████████| 51kB 3.2MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.33.0 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=f6c50bef23c68f6be4a0fac667e553f7b505157be7f4423cf5faedf9e28b42ce
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch
     |████████████████████████████████| 92kB 4.7MB/s 
     |████████████████████████████████| 686kB 12.7MB/s 
     |████████████████████████████████| 204kB 35.1MB/s 
  Created wheel for pytorch-toolbelt: filename=pytorch_toolbelt-0.3.2-cp36-none-any.whl size=122683 sha256=b92dcca0a72a940523f1a71da7baada1932df5e95fed8a3670481147c7e99544
  Stored in directory: /root/.cache/pip/wheels/02/8c/02/3f1

In [4]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 6.3MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.4 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [3]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils import *
#from simple_model import *
#from preprocess import *
#from make_labels import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy
from tqdm.auto import tqdm

alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


In [4]:
data = pd.read_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/splited_train_new.csv')
splits = pickle.load(open(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/cv_split.pickle', 'rb'))
labels = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
image_size = 240
fold_idx = 3
batch_size = 32
lr = 1e-5

In [5]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

num_wkr=0

In [6]:
train_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-train/'
valid_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-train/'

train_dataset1 = EyeDataset(dataset_path=train_path1,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))

train_loader1 = DataLoader(train_dataset1,
                          num_workers=num_wkr,
                          pin_memory=True,
                          batch_size=batch_size,
                          shuffle=False)
logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_clahe/'
modelA = prepare_model('efficientnet-b3', 8)
modelA.cuda()
modelA.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_clahe/checkpoints/best.pth'))['model_state_dict'])
modelA.eval()

Will load efficientnet-b3 with 8 classes
Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [7]:
train_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-train-full/'
valid_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-train-full/'

train_dataset2 = EyeDataset(dataset_path=train_path2,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))
train_loader2 = DataLoader(train_dataset2,
                          num_workers=num_wkr,
                          pin_memory=True,
                          batch_size=batch_size,
                          shuffle=False)

logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_vessel/'
modelB = prepare_model('efficientnet-b3', 8)
modelB.cuda()
modelB.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/initial_test_b3_fold3_vessel/checkpoints/best.pth'))['model_state_dict'])
modelB.eval()

Will load efficientnet-b3 with 8 classes
Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [ ]:

# x = modelA.extract_features(torch.ones((1,3,224,224)).cuda())
# # feature ngộ nghĩnh vl

# x1 = nn.functional.adaptive_avg_pool2d(x, 1).squeeze(-1).squeeze(-1)
# x2 = x1.clone()

# y = torch.cat((x1,x2), 1)

# y_sigmoid = torch.sigmoid(y)

RuntimeError: ignored

In [ ]:
n = nn.Linear(48, 24)
a = n(torch.rand(55, 64, 48))

a.flatten(start_dim=1).shape

torch.Size([55, 1536])

In [ ]:
X = torch.rand((1,3072))

W_x = nn.Parameter(torch.Tensor(48, 24))
b_x = nn.Parameter(torch.Tensor(1, 1536))
# print(W_x.shape)
# print(type(W_x))
# print(b_x.shape)

gate = torch.sigmoid((X.view(64, 48) @ W_x).flatten() + b_x)
# gate.shape
# Sigmoid(X * W_x + b_x)

# Sigmoid = 1x1536

# X ==> 1 x 3072
# X' => 64 x 48
# W_X ==> 48 x 24
# # b_x ==> 1x1536.

# x2 ==> 1x1536
# torch.arange(0, 1000).view(50, 20)

torch.Size([1, 1536])

In [ ]:
m = nn.Softmax(dim=1)
input = torch.randn(1,5)
output = m(input)
output.sum()

tensor(1.)

In [8]:
import sklearn
class NeuralNet(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=8):
        super(NeuralNet, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.drop = nn.Dropout(0.3)
        self.out2 = nn.Linear(1536, 8)
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()
        # self.W_x = nn.Parameter(torch.Tensor(48, 24)).to(device)
        # self.b_x = nn.Parameter(torch.Tensor(1, 1536)).to(device)
        self.W_g = nn.Linear(48, 24)
        # nn.init.xavier_uniform_(self.W_x.data)
        # nn.init.xavier_uniform_(self.b_x.data)
        nn.init.xavier_uniform_(self.W_g.weight)

    def forward(self, x1, x2):
        x1 = self.modelA.extract_features(x1)
        x2 = self.modelB.extract_features(x2)
        # print(x1.shape)
        x1 = nn.functional.adaptive_avg_pool2d(x1, 1).squeeze(-1).squeeze(-1)
        x2 = nn.functional.adaptive_avg_pool2d(x2, 1).squeeze(-1).squeeze(-1)
        # print('shape of x1:',x1.shape)
        # x1 = x1.view(1,1536)
        # x1 = torch.Tensor.cpu(x1).detach().numpy()
        # x2 = x2.view(1,1536)
        # x2 = torch.Tensor.cpu(x2).detach().numpy()
        x = torch.cat((x1, x2), 1)   
        # print('shape of feature cat:',x.shape)  
        # print(type(x))
        # sigmoid = torch.sigmoid(x)
        # multi = tf.math.multiply(sigmoid,x2)
        # add = multi + x1
        # add = add.numpy()
        # x = torch.from_numpy(add)
        # x = self.drop(x)
        # x = x.flatten()
        # x = self.out2(x)
        
        # gate = torch.sigmoid((x.view(64, 48) @ self.W_x).flatten() + self.b_x)
        gate = torch.sigmoid(self.W_g(x.view(-1, 64, 48))).flatten(start_dim=1)
        # print('value of gate:',gate)
        # print('shape of gate:',gate.shape)  
        features = gate * x2 + x1
        
        # print('value of features:',gate)
        # print('shape of features:',features.shape)  
        # features = features.cpu().detach().numpy()
        # features = torch.from_numpy(features)
        features = self.drop(features)
        features = self.out2(features)
        return features

In [ ]:
input1 = torch.rand([32, 3, 240, 240]).to(device)
input2 = torch.rand([32, 3, 240, 240]).to(device)

probabilities = []
x_model = NeuralNet(modelA, modelB)
x_model.to(device)
y = x_model(input1,input2)
print(y.shape)

softmax(y, dim=1)

RuntimeError: ignored

In [9]:
x_model = NeuralNet(modelA, modelB)
x_model.to(device)

NeuralNet(
  (modelA): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_p

In [10]:
!nvidia-smi

Fri Aug  7 02:47:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    60W / 149W |    498MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# freeze params in model A, model B
for param in x_model.modelA.parameters():
  param.requires_grad = False

for param in x_model.modelB.parameters():
  param.requires_grad = False

**TRAIN**

In [ ]:
optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5) ## em truyền params nó vào đây
criterion = nn.CrossEntropyLoss()
# scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=2)
loss_log = []
PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/clahe_vessel_forgetgate_new/'
if not os.path.exists(PATH_SAVE):
       os.makedirs(PATH_SAVE)

x_model.train()
for epoch in tqdm(range(20)):
  tqdm_iter = tqdm(zip(train_loader1,train_loader2))
  for i1, i2 in tqdm_iter:
    #print(i1)
    # stt1, data1 = i1
    input1,target1 = i1
    #print(i2)
    # stt2, data2 = i2
    input2,target2 = i2


    # zero the parameter gradients
    optimizer.zero_grad()
    input1, target1 = input1.to(device), target1.to(device)
    input2, target2 = input2.to(device), target2.to(device)

    # forward + backward + optimize
    outputs = x_model(input1,input2)
    #print(outputs)
    #print(target1)
    # outputs = outputs.view(1,8)

    loss = criterion(outputs, target1)
    loss.backward()
    tqdm_iter.set_postfix(loss=loss.item())
    optimizer.step() ## đây nó update đôn params kia. em muốn không train phần A hay B em phải chạy cái phần require_grad = false gì đấy, mà nhét cái đó vô chỗ nào? em muons làm gì, em muốn k train para trong model A,B
  loss_log.append(loss)
  torch.save(x_model.state_dict(), PATH_SAVE + 'clahe_vessel_forgetgate_new_epoch_'+str(epoch)+'.pth')

with open(PATH_SAVE+"clahe_vessel_forgetgate_new.txt", "w") as output:
    output.write(str(loss_log))

In [12]:
for i1, i2 in zip(train_loader1,train_loader2):
  print(i1,i2)
  break

[tensor([[[[-2.0494, -2.0837, -2.0323,  ..., -2.1008, -2.1179, -2.1008],
          [-2.0837, -2.0494, -2.0837,  ..., -2.1179, -2.1008, -2.0837],
          [-2.0837, -2.0494, -2.0494,  ..., -2.0494, -2.0837, -2.0837],
          ...,
          [-2.0837, -2.0494, -2.0494,  ..., -2.0837, -2.0152, -2.0494],
          [-2.0494, -2.0837, -2.1008,  ..., -2.0494, -2.0837, -2.0837],
          [-2.0837, -2.1179, -2.0837,  ..., -2.0152, -2.1179, -2.0152]],

         [[-1.9657, -2.0007, -1.9482,  ..., -2.0182, -2.0357, -2.0182],
          [-2.0007, -1.9657, -2.0007,  ..., -2.0357, -2.0182, -2.0007],
          [-2.0007, -1.9657, -1.9657,  ..., -1.9657, -2.0007, -2.0007],
          ...,
          [-2.0007, -1.9657, -1.9657,  ..., -2.0007, -1.9307, -1.9657],
          [-1.9657, -2.0007, -2.0182,  ..., -1.9657, -2.0007, -2.0007],
          [-2.0007, -2.0357, -2.0007,  ..., -1.9307, -2.0357, -1.9307]],

         [[-1.7347, -1.7696, -1.7173,  ..., -1.7870, -1.8044, -1.7870],
          [-1.7696, -1.7347, 

**TEST**

In [ ]:
x_model.load_state_dict(torch.load(os.path.join(
        '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs_final/clahe_vessel_forgetgate_new/clahe_vessel_forgetgate_new_epoch_18.pth')))
x_model.eval()

test_data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/XYZ_ODIR.csv')
test_data_left = test_data.copy()
test_data_right = test_data.copy()
test_data_left.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_left.jpg')
test_data_right.loc[:, 'id'] = test_data_left.ID.apply(
    lambda x: str(x)+'_right.jpg')
test_data = pd.concat([test_data_left, test_data_right])
test_data.sort_values(['ID'], inplace=True)
test_path = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/clahe-test/'
test_dataset = EyeDataset(dataset_path=test_path,
                          labels=test_data.loc[:, labels].values,
                          ids=test_data.loc[:, 'id'].values,
                          albumentations_tr=aug_val(image_size))
test_loader = DataLoader(test_dataset,
                         num_workers=num_wkr,
                         pin_memory=True,
                         batch_size=batch_size,
                         shuffle=False)


test_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/vessel-test-full/'
test_dataset_vessel = EyeDataset(dataset_path=test_path1,
                                 labels=test_data.loc[:, labels].values,
                                 ids=test_data.loc[:, 'id'].values,
                                 albumentations_tr=aug_val(image_size))
test_loader_vessel = DataLoader(test_dataset_vessel,
                                num_workers=num_wkr,
                                pin_memory=True,
                                batch_size=batch_size,
                                shuffle=False)

probabilities = []
probabilities_list = []
exp_name = 'initial_test_b3'
#optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5)

for i1, i2 in zip(enumerate(test_loader), enumerate(test_loader_vessel)):
    # print(i1)
    stt1, data1 = i1
    input1, target1 = data1
    # print(i2)
    stt2, data2 = i2
    input2, target2 = data2
    # zero the parameter gradients
    #optimizer.zero_grad()
    input1, target1 = input1.to(device), target1.to(device)
    input2, target2 = input2.to(device), target2.to(device)

    # forward + backward + optimize
    outputs = x_model(input1, input2)
    outputs = torch.Tensor.cpu(outputs).detach().numpy()
    probabilities = softmax(torch.from_numpy(outputs), dim=0).numpy()
    # print(probabilities)
    for idx in range(probabilities.shape[0]):
        if all(probabilities[idx] < 0.5):
            probabilities[idx][0] = 1.0
    probabilities_list.append(probabilities)
arr_list = []
arr = np.array(probabilities_list)
arr_list.append(arr)
print(arr_list)


probabilities_combined = np.stack(arr_list, axis=0).mean(axis=0)
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)
predicted_labels['id'] = test_data.loc[:, 'id'].values
predicted_labels.loc[:, 'ID'] = predicted_labels.id.apply(
    lambda x: x.split('_')[0])
predicted_labels_groupped = predicted_labels.groupby(
    ['ID']).aggregate(dict(zip(labels, ['max']*(len(labels)))))
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(
    int)
predicted_labels_groupped.reset_index(drop=True, inplace=True)
predicted_labels_groupped.sort_values('ID', inplace=True)
predicted_labels_groupped = predicted_labels_groupped.loc[:, ['ID']+labels]
predicted_labels_groupped.to_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/submit_final/clahe_vessel_forgetgate_new_epoch_19.csv', index=False)

In [ ]:
print(len(arr_list[0][0]))

32


In [ ]:
arr_list = []
arr = np.array(probabilities_list)
arr_list.append(arr)
print(arr_list)

In [ ]:
print(probabilities_list[1])

[[2.2144068e-04 4.7661279e-06 4.9535003e-03 9.9860054e-01 9.5265469e-04
  8.2829781e-03 3.3253014e-02 4.9108714e-02]
 [1.0000000e+00 1.3654106e-02 1.6948028e-02 1.9251675e-05 2.4395110e-04
  8.7407477e-02 1.9258418e-03 1.4824200e-04]
 [1.0000000e+00 2.3947457e-04 1.9015754e-03 2.9168342e-04 1.0105540e-02
  3.0838929e-03 9.4037894e-03 1.3055870e-02]
 [1.0000000e+00 3.0995693e-02 3.5283307e-03 5.4242759e-05 1.7945549e-03
  3.7506870e-03 6.2131430e-03 2.7398083e-03]
 [2.9051199e-04 2.8347517e-06 1.9345835e-02 2.3374027e-04 7.6764899e-01
  1.5555695e-03 2.5892500e-02 2.1067078e-01]
 [1.0000000e+00 1.9937640e-03 2.3203806e-03 1.7631479e-06 1.0871205e-01
  9.0088072e-04 1.4619349e-03 2.7955666e-03]
 [1.0000000e+00 1.0435772e-03 1.9407854e-03 4.4337203e-06 7.6591113e-04
  2.8260378e-03 9.9492224e-04 2.6505769e-03]
 [1.0000000e+00 4.7300367e-05 4.6464153e-02 9.5620062e-06 8.1378054e-03
  8.2513308e-03 1.5272426e-03 9.0417603e-04]
 [1.0000000e+00 3.6882999e-04 2.8758939e-02 3.6517076e-06 1.9461

In [ ]:
arr_list1 = []
for i in probabilities_list:
  for j in i:
    print(j)
    #print(len(j))
    #x = np.array(j)
    arr_list1.append(x)
#print(arr_list1)


[1.0000000e+00 1.6316134e-04 5.2866463e-02 4.8119966e-03 5.8490634e-05
 8.8020734e-02 1.1512419e-03 2.4260061e-03]
[1.0000000e+00 7.7522993e-03 3.0436271e-03 7.2792908e-03 2.1664324e-04
 3.7784681e-01 6.8974728e-03 7.4463210e-04]
[1.0000000e+00 8.0106091e-03 8.5256775e-05 1.9860677e-03 5.4553780e-03
 1.7367763e-03 4.7698678e-03 1.0894294e-02]
[1.0000000e+00 1.7049800e-01 1.5615511e-03 1.2017510e-02 3.5965994e-03
 4.9334257e-03 3.1994242e-02 5.7079447e-05]
[1.0000000e+00 6.7672564e-04 1.0034190e-03 2.2619946e-02 6.7500259e-05
 1.3488573e-04 2.1234720e-03 2.4030743e-02]
[1.0000000e+00 8.5264684e-05 1.2379026e-03 5.7227097e-02 1.4787121e-03
 1.2727933e-03 8.6145833e-02 3.7907790e-02]
[1.00000000e+00 5.87351969e-04 3.00790387e-04 6.64554723e-03
 1.77870702e-03 9.49199457e-05 8.40691268e-04 1.15386015e-02]
[1.0000000e+00 5.7576858e-03 2.7080756e-04 3.6181763e-03 3.3660306e-04
 5.2929803e-04 1.0146903e-03 3.5586834e-04]
[1.0000000e+00 3.5348665e-02 4.0055756e-03 1.4681787e-02 3.3683394e-04
 

In [ ]:
probabilities_combined = np.stack(arr_list, axis=0).mean(axis=0)
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)
predicted_labels['id'] = test_data.loc[:, 'id'].values
predicted_labels.loc[:, 'ID'] = predicted_labels.id.apply(
    lambda x: x.split('_')[0])
predicted_labels_groupped = predicted_labels.groupby(
    ['ID']).aggregate(dict(zip(labels, ['max']*(len(labels)))))
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(
    int)
predicted_labels_groupped.reset_index(drop=True, inplace=True)
predicted_labels_groupped.sort_values('ID', inplace=True)
predicted_labels_groupped = predicted_labels_groupped.loc[:, ['ID']+labels]
predicted_labels_groupped.to_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/submit_final/clahe_vessel_forgetgate_new_epoch_18.csv', index=False)

ValueError: ignored

In [ ]:
for idx in range((probabilities.shape[0])):#32
  if all(probabilities[idx] < 0.5):
    print(probabilities[idx])


[4.1628491e-02 1.6316134e-04 5.2866463e-02 4.8119966e-03 5.8490634e-05
 8.8020734e-02 1.1512419e-03 2.4260061e-03]
[1.9674080e-03 7.7522993e-03 3.0436271e-03 7.2792908e-03 2.1664324e-04
 3.7784681e-01 6.8974728e-03 7.4463210e-04]
[3.8555430e-04 8.0106091e-03 8.5256775e-05 1.9860677e-03 5.4553780e-03
 1.7367763e-03 4.7698678e-03 1.0894294e-02]
[2.5963722e-05 1.7049800e-01 1.5615511e-03 1.2017510e-02 3.5965994e-03
 4.9334257e-03 3.1994242e-02 5.7079447e-05]
[2.6914554e-02 6.7672564e-04 1.0034190e-03 2.2619946e-02 6.7500259e-05
 1.3488573e-04 2.1234720e-03 2.4030743e-02]
[4.3647902e-04 8.5264684e-05 1.2379026e-03 5.7227097e-02 1.4787121e-03
 1.2727933e-03 8.6145833e-02 3.7907790e-02]
[2.24616632e-01 5.87351969e-04 3.00790387e-04 6.64554723e-03
 1.77870702e-03 9.49199457e-05 8.40691268e-04 1.15386015e-02]
[4.6114126e-01 5.7576858e-03 2.7080756e-04 3.6181763e-03 3.3660306e-04
 5.2929803e-04 1.0146903e-03 3.5586834e-04]
[0.00053047 0.03534866 0.00400558 0.01468179 0.00033683 0.0018258
 0.018